(1) Collect sample data form Google News with keyword "ChatGPT" snd "Diffussion models", save them into a Excel file as first colum

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_title(url, block_element, cls, num):
    # Send a request to the search page with given url
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    mycls = soup.find_all(block_element, class_= cls)
    i = 1
    title_list = []
    for element in mycls:
        if i <= num:
            title_list.append(element.text) 
            i += 1
        else:
            break
    return title_list

In [3]:
gpt_list = get_title('https://news.google.com/search?q=ChatGPT&hl=en-US&gl=US&ceid=US%3Aen', 
                     'a', 'DY5T1d RZIKme', 10)
dm_list = get_title('https://news.google.com/search?q=Diffusion%20models&hl=en-US&gl=US&ceid=US%3Aen', 
                    'a', 'DY5T1d RZIKme', 10)

In [4]:
title_list = gpt_list + dm_list

In [5]:
title_list

['300 million jobs could be affected by latest wave of AI, says Goldman Sachs',
 'In Sudden Alarm, Tech Doyens Call for a Pause on ChatGPT',
 'AI Leaders Urge Labs to Halt Training Models More Powerful Than ChatGPT-4',
 'Musk, scientists call for halt to AI race sparked by ChatGPT',
 "Japan PM Quizzed by ChatGPT as Lawmaker Enlists Bot's Help",
 'ChatGPT Made Me Cry and Other Adventures in AI Land',
 'AI ChatGPT Chatbot Related Prompt Engineer Jobs Pay Up to $335,000',
 'Google denies Bard was trained with ChatGPT data',
 'How ChatGPT and Bard Performed as My Executive Assistants',
 'ChatGPT vs. Google Bard vs. Bing Chat: Which generative AI solution is best?',
 'Diffusion models can be contaminated with backdoors, study finds',
 'Meet Web Stable Diffusion: An AI Project That Brings Stable Diffusion Models To Web Browsers',
 'Meet Pix2Video: A Training-Free And Text-Guided AI Approach That Simplifies Video Editing Using Image Diffusion Models',
 'Researchers From Stanford Introduce Loc

(2) Organize sentiment score from Afinn, Word2Vec, DistilBert and save as second column

In [9]:
# Get sentiment analysis of each title by using AFINN
from afinn import Afinn
afinn = Afinn()
afinn_score_list = [afinn.score(title) for title in title_list]

In [15]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [16]:
# Get only the English sentiment analysis from Hugging Face's model hub
pre_model = "textattack/distilbert-base-uncased-imdb"
# Using DistilBert model
tokenizer = DistilBertTokenizer.from_pretrained(pre_model)
model = DistilBertForSequenceClassification.from_pretrained(pre_model)

In [17]:
def calc_sentiment_score(title):
    # Tokenize the title and convert to tensors
    inputs = tokenizer(title, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Pass the input through the DistilBert model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Calculate the sentiment score
    sentiment_score = torch.softmax(logits, dim=1).numpy()[0]

    return sentiment_score

# Calculate sentiment scores for each title
DistilBert_senti = []
for title in title_list:
    sentiment_score = calc_sentiment_score(title)
    DistilBert_senti.append("Nagetive" if sentiment_score[0] > sentiment_score[1] else "Postive")

In [18]:
DistilBert_senti

['Nagetive',
 'Nagetive',
 'Nagetive',
 'Nagetive',
 'Nagetive',
 'Postive',
 'Nagetive',
 'Nagetive',
 'Nagetive',
 'Nagetive',
 'Nagetive',
 'Postive',
 'Postive',
 'Postive',
 'Postive',
 'Postive',
 'Postive',
 'Postive',
 'Postive',
 'Postive']

In [19]:
# Combine affin score and DistilBert_senti as 1 column
combined_list = [f"{afinn_score}/{DistilBert}" for afinn_score, DistilBert in zip(afinn_score_list, DistilBert_senti)]

In [20]:
combined_list

['-1.0/Nagetive',
 '-2.0/Nagetive',
 '2.0/Nagetive',
 '0.0/Nagetive',
 '2.0/Nagetive',
 '1.0/Postive',
 '-1.0/Nagetive',
 '-2.0/Nagetive',
 '0.0/Nagetive',
 '4.0/Nagetive',
 '-2.0/Nagetive',
 '4.0/Postive',
 '1.0/Postive',
 '0.0/Postive',
 '0.0/Postive',
 '2.0/Postive',
 '0.0/Postive',
 '0.0/Postive',
 '0.0/Postive',
 '0.0/Postive']

(3) Combine Column1 & Column2, Export into Excel file

In [23]:
import pandas as pd
df = pd.DataFrame({"News Title": title_list, "Sentiment Score(Affin/DistilBert)": combined_list})


In [27]:
from google.colab import files
file_name = "Sentiment.xls"
df.to_excel(file_name, engine="openpyxl")
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>